In [ ]:
import numpy as np
import utilities as u

from scipy.io import loadmat
from datetime import datetime
from pathlib import Path

# Split the SWAN matlab file
This script allows to convert the SWAN matlab file into a series of csv files for each time step (and sea state variable)

In [ ]:
# Definition of file names and constants

# Define the type of the time step (hrs, min) and the size of it
time_step_type = "hrs" # could be min too
time_step = 1          # can be something like 20 (for minutes)  

# Define the data time span
start_date = datetime(2018, 1, 1, 0, 0)
end_date = datetime(2019, 12, 31, 23, 0)

# Variable to extract (has to be the name given by SWAN)
var = "Hsig"
# How the variable name will appear in the output
var_short = "Hs"

# Path of the Matlab file to be split
fn = "../SWAN/out/BaskCoast_ZoomBtz.mat"

# Path of the output files, can be for reference (HR) or input (LR) files
#dirout = f"Data/LR/DS_RAW/Kernel_16/{var_short}/"
dirout = Path(f"Data/HR/{var_short}")

# Prefix of the output files
prefix = "BaskCoast_{}_{}.npy"

# Check if output folder(s) exists, if not, create it/them
if not dirout.exists():
    dirout.mkdir(parents=True)

## Load whole file into RAM
This is the easiest way if the whole data set fits in RAM

In [ ]:
# Load Matlab File
data = loadmat(fn)

# Initialize a date generator to loop over the dates
daterange = u.create_dategenerator(time_step, time_step_type)

for i, date in enumerate(daterange(start_date, end_date)):
    # Get the right key for the according day and hour
    date = date.strftime('%Y%m%d_%H%M00')
    file_key = var + "_" + date

    # Access the array and save it in a file
    fn_out = dirout / prefix.format(var_short.upper(), i+1)
    np.save(fn_out, data[file_key][1:,1:])

## Load each time step one by one
This is only recommended if the data set does not fit in RAM.
The code is not optimized and thus very slow.

In [ ]:
for i, date in enumerate(daterange(start_date, end_date)):
    # Get the right key for according day and hour
    date = date.strftime('%Y%m%d_%H%M00')
    file_key = var + "_" + date

    data = loadmat(fn, variable_names=(file_key))
    
    # Access the array and save it in a file
    fn_out = dirout / prefix.format(var_short.upper(), i+1)
    np.save(fn_out, data[file_key][1:,1:])